In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - mask_essf.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Create a mask for the effective survey selection function grid which removes 
any fields where data is not considered for modelling.
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, copy, dill as pickle

## Matplotlib
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu

# Set APOGEE version for the package
apogee_results_vers = 'l33'
# Year 7 is appropriate for DR16 (l33)
apo_combined_select_year = 7
os.environ['RESULTS_VERS'] = apogee_results_vers

import apogee.select as apsel
import apogee.tools as apotools

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import util as putil
from ges_mass import mass as pmass

### Scale parameters
ro = 8.
vo = 220
zo = 0.0208 # Bennett+ 2019

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Preliminaries

In [ ]:
# Pathing
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    = putil.parse_config_dict(cdict,keywords)

### Load data

In [ ]:
# Filenames
apogee_SF_filename = data_dir+gaia_apogee_dir+'apogee_SF.dat'
apogee_effSF_filename = data_dir+gaia_apogee_dir+\
    'apogee_effSF_grid_inclArea.dat'

# Selection function
print('APOGEE data release is: '+apogee_dr+', and results version is: '+apogee_results_vers)
if os.path.exists(apogee_SF_filename):
    print('Loading APOGEE selection function from '+apogee_SF_filename)
    with open(apogee_SF_filename, 'rb') as f:
        apogee_SF = pickle.load(f)
    ##wi
else:
    sys.exit('Could not find APOGEE selection function, make it. Exiting...')

# Load the effective selection function grid
if os.path.exists(apogee_effSF_filename):
    print('\nLoading effective selection function from '+apogee_effSF_filename)
    with open(apogee_effSF_filename,'rb') as f:
        apogee_effSF_grid_inclArea = pickle.load(f)
    ##wi
else:
    sys.exit('\nFailed to load effective survey section function')

### Field and eSSF grid information

In [ ]:
# Field info
apogee_field = apogee_SF._apogeeField
n_field = len(apogee_field)
field_glon = apogee_field['GLON']
field_glat = apogee_field['GLAT']
field_locid = apogee_field['LOCATION_ID']

# Distance modulus grid
dmod,dist = putil.make_dmod_grid(ndmod,dmod_min,dmod_max)

# Some sanity checks
assert len(field_glon) == apogee_effSF_grid_inclArea.shape[0]
assert np.all(field_locid == apogee_SF._locations)

### Show the effective selection function
Plot fields with and without the effective selection function. There should be 
some fields for which the effective selection function is zero along the entire 
length of the distance modulus grid (second axis of the effective selection 
function grid). This is set to an array of zeros in `make_effsf.py` when there 
are no spectroscopic targets in the field for any cohort.

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)

plot_glon = copy.deepcopy(field_glon)
plot_glat = copy.deepcopy(field_glat)
plot_glon[plot_glon > 180] = plot_glon[plot_glon > 180] - 360

no_effSF = np.sum(apogee_effSF_grid_inclArea == 0., axis=1) == ndmod

ax.scatter(plot_glon[~no_effSF], plot_glat[~no_effSF], 
           c='ForestGreen', s=4, zorder=2)
ax.scatter(plot_glon[no_effSF], plot_glat[no_effSF], 
           c='Red', s=8, zorder=1)

ax.set_xlabel(r'$\ell$ [deg]')
ax.set_ylabel(r'$b$ [deg]')
ax.set_xlim(-180,180)
ax.set_ylim(-90,90)
ax.invert_xaxis()

fig.show()

### Now mask out fields where data is not considered

This includes:
1. Fields within a 20 degree square centered on the galactic bulge
2. Fields with a globular cluster in them
3. Fields with no spectroscopic targets

In [ ]:
# Cut bulge fields. Within 20 degrees of the galactic center
mask_bulge = ~(((field_glon > 340.) |\
                (field_glon < 20.)) &\
               (np.fabs(field_glat) < 20.)
              )

# Cut fields containing enhancements of globular cluster stars
gc_locid = [2011,4353,5093,5229,5294,5295,5296,5297,5298,5299,5300,5325,5328,
            5329,5438,5528,5529,5744,5801]
mask_gc = ~np.isin(field_locid,gc_locid)

# Mask parts of the selection function where there are no spectroscopic 
# targets and the whole field is set to zero.
mask_nspec = ~np.all(apogee_effSF_grid_inclArea < 1e-9, axis=1)
        
# The fitting mask, which represents the parts of the effective selection 
# function grid which are used in the fitting process.
mask_effSF = mask_bulge & mask_gc & mask_nspec

### Show the masks

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)

plot_glon = copy.deepcopy(field_glon)
plot_glat = copy.deepcopy(field_glat)
plot_glon[plot_glon > 180] = plot_glon[plot_glon > 180] - 360

ax.scatter(plot_glon[~mask_bulge], plot_glat[~mask_bulge], 
           s=8, c='Red', zorder=1)
ax.scatter(plot_glon[mask_bulge], plot_glat[mask_bulge], 
           c='ForestGreen', s=4, zorder=2)

ax.plot([20,20],[20,-20],c='Black',linewidth=1.,zorder=3)
ax.plot([20,-20],[-20,-20],c='Black',linewidth=1.,zorder=3)
ax.plot([-20,-20],[-20,20],c='Black',linewidth=1.,zorder=3)
ax.plot([-20,20],[20,20],c='Black',linewidth=1.,zorder=3)

ax.set_xlabel(r'$\ell$ [deg]')
ax.set_ylabel(r'$b$ [deg]')
ax.set_xlim(-180,180)
ax.set_ylim(-90,90)
ax.invert_xaxis()

fig.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)

plot_glon = copy.deepcopy(field_glon)
plot_glat = copy.deepcopy(field_glat)
plot_glon[plot_glon > 180] = plot_glon[plot_glon > 180] - 360

ax.scatter(plot_glon[~mask_gc], plot_glat[~mask_gc], s=8, c='Red', 
           zorder=1)
ax.scatter(plot_glon[mask_gc], plot_glat[mask_gc], 
           c='ForestGreen', s=4, zorder=2)

ax.set_xlabel(r'$\ell$ [deg]')
ax.set_ylabel(r'$b$ [deg]')
ax.set_xlim(-180,180)
ax.set_ylim(-90,90)
ax.invert_xaxis()

fig.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)

plot_glon = copy.deepcopy(field_glon)
plot_glat = copy.deepcopy(field_glat)
plot_glon[plot_glon > 180] = plot_glon[plot_glon > 180] - 360

ax.scatter(plot_glon[~mask_nspec], plot_glat[~mask_nspec], s=8, c='Red', 
           zorder=1)
ax.scatter(plot_glon[mask_nspec], plot_glat[mask_nspec], 
           c='ForestGreen', s=4, zorder=2)

ax.set_xlabel(r'$\ell$ [deg]')
ax.set_ylabel(r'$b$ [deg]')
ax.set_xlim(-180,180)
ax.set_ylim(-90,90)
ax.invert_xaxis()

fig.show()

Note that many fields overlap with fields containing globular clusters yet they 
may not contain significant numbers of globular clusters themselves.

### Save the masks

In [ ]:
mask_effSF_filename = data_dir+gaia_apogee_dir+\
    'apogee_effSF_grid_mask.npy'
np.save(mask_effSF_filename,mask_effSF)

# Also save one which doesn't exclude globular cluster fields
mask_effsf_no_gc_filename = data_dir+gaia_apogee_dir+\
    'apogee_effSF_grid_mask_no_gc.npy'
# np.save(mask_effSF_no_gc_filename,mask_bulge & mask_nspec)